In [2]:
from parcels import Field,FieldSet,ParticleSet,Variable,JITParticle,AdvectionRK4,ErrorCode,VectorField

import datetime
from datetime import timedelta as delta 
from glob import glob

import math as m
from operator import attrgetter

import numpy as np
from netCDF4 import Dataset

from scipy.interpolate import griddata


## Function and Kernel

In [2]:
##Function and Kernels

class PlasticParticle(JITParticle):  
    # Define a new particle class that contains three extra variables
    beached = Variable('beached', dtype =np.float32, initial = 0 )
    
def Beached(particle, fieldset, time):
    """
    Kernel to calculate the number of stranded particles
    """
    (u,v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
    
    #u,v never really = 0, instead use a really small number
    if fabs(u) <= 1e-20 and fabs(v) <= 1e-20 : 
        particle.beached = 1       #beached = 1, sea = 0
        

def set_fieldset(stokes, currents, fieldset):
    """
    Kernel to set the fieldset: Stokes drift added only off-coasts
    """
    
    (u,v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]

    if fabs(u) <= 1e-20 and fabs(v) <= 1e-20 : 
        fset = FieldSet(U = u, V  = v)
    elif fabs(u) > 1e-20 and fabs(v) > 1e-20: 
        fset = FieldSet(U = u + u_uss, V = v + v_uss)

        
    return fset

def DiffusionM1(particle, fiedlset, time):
    """ 
    Parcels implemented kernel (AdvectionRK4DiffusionM1):
    """
    if particle.beached == 0 :

        # RK4 terms
        (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        lon1, lat1 = (particle.lon + u1 * .5 * particle.dt, particle.lat + v1 * .5 * particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1]
        lon2, lat2 = (particle.lon + u2 * .5 * particle.dt, particle.lat + v2 * .5 * particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2]
        lon3, lat3 = (particle.lon + u3 * particle.dt, particle.lat + v3 * particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3]

        # Wiener increment with zero mean and std of sqrt(dt)
        dWx = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)
        dWy = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)

        Kxp1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon + fieldset.dres]
        Kxm1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon - fieldset.dres]
        dKdx = (Kxp1 - Kxm1) / (2 * fieldset.dres)
        bx = math.sqrt(2 * fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon])

        Kyp1 = fieldset.Kh_meridional[time, particle.depth, particle.lat + fieldset.dres, particle.lon]
        Kym1 = fieldset.Kh_meridional[time, particle.depth, particle.lat - fieldset.dres, particle.lon]
        dKdy = (Kyp1 - Kym1) / (2 * fieldset.dres)
        by = math.sqrt(2 * fieldset.Kh_meridional[time, particle.depth, particle.lat, particle.lon])

        # Particle positions are updated only after evaluating all terms.
        particle.lon += ((u1 + 2 * u2 + 2 * u3 + u4) / 6.) * particle.dt + 0.5 * dKdx * (dWx**2 + particle.dt) + bx * dWx
        particle.lat += ((v1 + 2 * v2 + 2 * v3 + v4) / 6.) * particle.dt + 0.5 * dKdy * (dWy**2 + particle.dt) + by * dWy


def periodicBC(particle, fieldset, time):
    """
    Kernel for periodic values in longitude
    """
    if particle.lon < 0.:
        particle.lon += 360.
    elif particle.lon >= 360.:
        particle.lon -= 360.
        
def OutOfBounds(particle, fieldset, time):
    particle.delete()
    
def set_currents (uvfiles):
    """
    Surface currents only
    """
    #all in the same file (CMEMS reanalysis)
    filenames = {'U': uvfiles,
            'V': uvfiles}
    dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
    variables = {'U':'uo', 'V' : 'vo'}
    uvindices = {'lon': range(2484, 2664), 'lat': range(1452, 1536)} #define domain 
   
    
    fset_currents = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation = False, 
                                    indices = uvindices)
    fset_currents.add_periodic_halo(zonal=True, meridional=False, halosize=5)
    
    return fset_currents

def set_stokes(stokesfiles, fieldset):
    """
    Kernel to add Stokes drift to the surface currents
    """
    stokesfilenames = {'Uuss': stokesfiles,
            'Vuss': stokesfiles}
    stokesdimensions = {'lon': 'longitude','lat': 'latitude', 'time': 'time'}
    stokesvariables ={'Uuss': 'uuss', 'Vuss': 'vuss'}
    
    stokesindices = {'lon': range(420, 440), 'lat': range(237, 253)} #define domain    
    
    Uuss = Field.from_netcdf(stokesfiles, ('Uuss', 'uuss'), stokesdimensions, fieldtype='U', 
                            allow_time_extrapolation=False,indices = stokesindices)
    Vuss = Field.from_netcdf(stokesfiles, ('Vuss', 'vuss'), stokesdimensions, fieldtype='V', 
                            allow_time_extrapolation=False, 
                            grid=Uuss.grid, dataFiles=Uuss.dataFiles, indices = stokesindices)
    
    fieldset.add_field(Uuss)
    fieldset.add_field(Vuss)
    uv_uss = VectorField('UVuss', fieldset.Uuss, fieldset.Vuss)
    fieldset.add_vector_field(uv_uss)


def BrownianMotion(particle, fieldset, time): #from Erik
    if beached == 0 : 
        kh_zonal = fieldset.Kh / math.pow(1000. * 1.852 * 60. * math.cos(particle.lat * math.pi / 180), 2)
        kh_meridional = fieldset.Kh / math.pow(1000.0 * 1.852 * 60.0, 2)

        r = 1/3.
        particle.lat += random.uniform(-1., 1.)*math.sqrt(2*math.fabs(dt)*kh_meridional/r)
        particle.lon += random.uniform(-1., 1.)*math.sqrt(2*math.fabs(dt)*kh_zonal/r)

def particles_grid(name, spacing = 0.15, minlon = 27, maxlon = 42, minlat = 41.5, maxlat = 47):
    """
    Kernel to create an uniform grid of particles for initial positions
    @author: David Wichmann, modified by Deborah Bassotto
    """
    
    griddir = r'/data/oceanparcels/input_data/NEMO-MEDUSA/ORCA0083-N006/domain/bathymetry_ORCA12_V3.3.nc'
    outdir= '/science-nfs-sys/vsm01/users/6312454/' + 'BS_grid'
    
    filename = griddir
    data = Dataset(filename,'r')
    bathy=np.array(data['Bathymetry'])
    lon=np.array([data['nav_lon']][0])
    lat=np.array([data['nav_lat']][0])
    
    grid=np.mgrid[minlon:maxlon:spacing,minlat:maxlat:spacing]
    n=grid[0].size;
    lons=np.reshape(grid[0],n)
    lats=np.reshape(grid[1],n)
      
    points = np.array([lon.flatten(), lat.flatten()]).T
    values = bathy.flatten()
    xi = (lons, lats)
    
    bathy_p = griddata(points, values, xi, method='nearest')
    
    
    Lons = np.array([lons[i] for i in range(len(lons)) if bathy_p[i] >17])
    Lats = np.array([lats[i] for i in range(len(lats)) if bathy_p[i] >17])
    
    Lons[Lons<0.] += 360.
    np.save(outdir + '_Longitude_' + str(name),Lons)
    np.save(outdir + '_Latitude_' + str(name),Lats)

    return Lons, Lats
    

## Loading Datasets

In [ ]:
##Data
outdir= '/science-nfs-sys/vsm01/users/6312454/' + 'BS_grid'
datadir = '/data/oceanparcels/input_data'

stokesfiles = sorted(glob(datadir+'/WaveWatch3data/CFSR/WW3-GLOB-30M_*.nc'))
uvfiles = sorted(glob(datadir +'/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/mercatorglorys12v1_gl12_mean_*.nc'))

##Coordinates particles grid
name = '05'
lons,lats = particles_grid(name)


print('Number of initial particles'+ '\t' + str(len(lons)))

## Creating fieldset

In [8]:
##Without Stokes drift
currents = set_currents(uvfiles)
fset = FieldSet(U = currents.U, V = currents.V)

##With Stokes drift
stokes = set_stokes(stokesfiles, fset)
fset_stokes = FieldSet(U =currents.U + stokes.U, V = currents.V + stokes.V)


## Adding Field for diffusion

From [Gräwe (2011)](https://www.sciencedirect.com/science/article/abs/pii/S1463500310001484?via%3Dihub)$\;$ (or check [Parcels Diffusion Tutorial](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_diffusion.ipynb)).

<br>

$ K_{meridional}(y) = \bar{K}\frac{2(1+\alpha)(1+2\alpha)}{\alpha^2 H^{1+1/\alpha}} \begin{cases}y(L-2y)^{1/\alpha}\;\textrm{and}\; 0 \leq y \leq L/2, \\ (L-y)(2y-1)^{1/\alpha}\; \textrm{and} \; H/2 \leq y \leq L \end{cases}$.


<br>
NB : If the dataset as values for Kh, this can be skipped.

In [9]:
size2D = (fset.U.grid.ydim, fset.U.grid.xdim)
KH_z = 0.25
alpha = 0.1
L = fset.U.grid.ydim
beta = np.zeros(fset.U.grid.ydim)
y_K = fset.V.grid.lat

for yi in range(len(y_K)):
    if y_K[yi] < L/2:
        beta[yi] = y_K[yi]*np.power(L - 2*y_K[yi], 1/alpha)
    elif y_K[yi] >= L/2:
        beta[yi] = (L - y_K[yi])*np.power(2*y_K[yi] - L, 1/alpha)

KH_m = KH_z*(2*(1+alpha)*(1+2*alpha))/(alpha**2*np.power(L, 1+1/alpha))*beta
Kh_meridional = np.array([KH_m for x in range(190)]).T


fset.add_field(Field('Kh_zonal', data=KH_z*np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset.add_field(Field('Kh_meridional', data=Kh_meridional*np.ones(size2D), 
                     #data=8 * np.ones(size2D),
                     lon=fset.U.grid.lon,lat=fset.U.grid.lat,mesh='spherical', allow_time_extrapolation=True))

fset.add_constant('dres', 0.000002)


### Advection

In [ ]:
## Initial uniform particles position over the Black-Sea
startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]


rundays = 365   #[days]
dt = 30         #[min]
nperloc = 20    #Number of particle released per location

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

#Uncomment if you want to release new partciles every x-time
#repeatdt = delta(days=30) 

psetname ='RUN4_test'

name = str(psetname)+str(rundays)+'d'  #outputfile name


pset_RUN4 = ParticleSet(fieldset=fset, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                        lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc])
                        #,repeatdt = repeatdt
                       )

kernels = pset_RUN4.Kernel(AdvectionRK4) + periodicBC + DiffusionM1 + Beached + StokesDrag

##Executing the simulation
pset_RUN4.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                    output_file
                  = pset_RUN4.ParticleFile(name = name, outputdt=delta(hours=12)), 
                    recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds},verbose_progress =False)